In [1]:
import datasetdatabase as dsdb
import pandas as pd
import quilt

In [2]:
mngr = dsdb.ConnectionManager(dsdb.LOCAL, user="jacksonb")
mngr

LOCAL:
	driver: sqlite
	link: /active/examples/local_database/local.db

In [3]:
local = mngr.connect(dsdb.LOCAL)
local

Recent Datasets:

In [4]:
quilt.ls()

/home/jovyan/.local/share/QuiltCli/quilt_packages


In [5]:
test = pd.DataFrame([{"hello": 1, "world": 2}, {"hello": 3, "world": 4}])
test

,hello,world
0,1,2
1,3,4


In [6]:
local.upload_dataset(test)

Creating Iota...
[============================================================] 100.0% (4/4) ~ 0:00:00 remaining
Creating Junction Items...


{'DatasetId': 1,
 'Name': 'fms_ae47a461a4ab791aab0fa25745ea846b',
 'Description': None,
 'SourceId': 1,
 'Created': '2018-07-10 21:42:11.535332'}

In [7]:
local

Recent Datasets:
{'DatasetId': 1, 'Name': 'fms_ae47a461a4ab791aab0fa25745ea846b', 'Description': None, 'SourceId': 1, 'Created': '2018-07-10 21:42:11.535332'}

In [8]:
quilt.ls()

/home/jovyan/.local/share/QuiltCli/quilt_packages
dsdb_storage/fms_ae47a461a4ab791aab0fa25745ea846b latest               5a7e03c6dc595b962ab822ef86cfc88b94ac79593e41abadb4e8960ab70b2ad3


In [9]:
# no duplication of iota, dataset, or source
local.upload_dataset(test)

{'DatasetId': 1,
 'Name': 'fms_ae47a461a4ab791aab0fa25745ea846b',
 'Description': None,
 'SourceId': 1,
 'Created': '2018-07-10 21:42:11.535332'}

In [10]:
quilt.ls()

/home/jovyan/.local/share/QuiltCli/quilt_packages
dsdb_storage/fms_ae47a461a4ab791aab0fa25745ea846b latest               5a7e03c6dc595b962ab822ef86cfc88b94ac79593e41abadb4e8960ab70b2ad3


In [11]:
local._deep_print()

------------------------------- DATASET DATABASE -------------------------------
--------------------------------------------------------------------------------
User:
rows: 1
example: {'UserId': 1, 'Name': 'jacksonb', 'Description': None, 'Created': '2018-07-10 21:42:11.422380'}
--------------------------------------------------------------------------------
Iota:
rows: 4
example: {'IotaId': 1, 'GroupId': 0, 'Key': 'hello', 'Value': '1', 'ValueType': "<class 'numpy.int64'>", 'Created': '2018-07-10 21:42:11.460891'}
--------------------------------------------------------------------------------
SourceType:
rows: 2
example: {'SourceTypeId': 1, 'Name': 'FileSource', 'Description': 'Id attached should be read using FMS get.', 'Created': '2018-07-10 21:42:07.917735'}
--------------------------------------------------------------------------------
Source:
rows: 1
example: {'SourceId': 1, 'SourceTypeId': 1, 'Name': 'fms_ae47a461a4ab791aab0fa25745ea846b', 'Created': '2018-07-10 21:42:11.4350

In [12]:
local.database.table("Run").get().all()

[{'RunId': 1,
  'AlgorithmId': 1,
  'UserId': 1,
  'Name': None,
  'Description': 'datasetdatabase.core.datasetdatabase._upload_dataset run for jacksonb',
  'Begin': '2018-07-10 21:42:11.432100',
  'End': '2018-07-10 21:42:11.601939'}]